In [30]:
from time import sleep

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
import selenium.webdriver.support.expected_conditions as EC
import funclib.baselib as baselib
import funclib.iolib as iolib
import pickle

logfile = 'c:/temp/jupyter/sa_scrape.log'
links_pickle = 'c:/temp/jupyter/links.pkl'
log = []
errs = []
print('Done')

Done


In [33]:
browser = webdriver.Firefox()
url = 'https://www.actionlibrary.com/cgi-bin/emapwww'
browser.get(url)

In [45]:
wait = WebDriverWait(browser, 15)
def wait_name(s):
    return wait.until(EC.presence_of_element_located((By.NAME, s)))

def wait_class_name(s):
    return wait.until(EC.presence_of_element_located((By.CLASS_NAME, s)))

def wait_link(s):
    return wait.until(EC.presence_of_element_located((By.LINK_TEXT, s)))

def wait_xpath(s):
    return wait.until(EC.presence_of_element_located((By.XPATH, s)))

In [37]:
usr = wait_name('_LGIN_Username')
pw = wait_name('_LGIN_Password')
sleep(1)
usr.send_keys('patricia')
sleep(1)
pw.send_keys('patricia1')
sleep(1)
login_button = wait_class_name('buttonstandard')
login_button.click()
sleep(5)

In [ ]:
#Load Up Feature Search and Search for Sea Angler

In [40]:
browser.switch_to.default_content()
browser.switch_to.frame('emapwww_Navigation')
sleep(1)
a_search = wait_link('Feature Search')
a_search.click()
sleep(1)
browser.switch_to.default_content()
browser.switch_to.frame("emapwww_MainSection")
print('waiting for [editionsinfo]publicationText...')
tb_sa = wait_name('[editionsinfo]publicationText')
print(tb_sa)
tb_sa.send_keys('Sea Angler')
sleep(1)




waiting for [editionsinfo]publicationText...
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="e9d028c6-e3a4-4839-9378-a07eb15fc51b", element="268b2975-7389-4c02-858b-c7eb35448444")>


In [47]:
btn_search = wait_xpath('//input[@value="Search"]')
btn_search.click()

In [39]:
#Setup some wait types

In [48]:
# 1 to 17025

def get_links(elements):
    '''get the four page links'''
    out = [x.get_attribute('href') for x in elements]
    return out


class ElementHasChanged():
    SCRAPED_LINKS = []
    
    def __init__(self, locator):
        self.locator = locator
        
    def __call__(self, browser):
        s = '//a[contains(@href, "emapwww?Page=DoPageSearch&Action=EditionPages&EditionURN=")]'
        try:
            elements = browser.find_elements_by_xpath(s)
        except Exception as e:
            was_err = True
        
        if elements:
            lnks = get_links(elements)
            was_err = all([s in ElementHasChanged.SCRAPED_LINKS for s in lnks])
        else:
            was_err = True
        
        if was_err:
            return False
        else:
            return elements
        

In [30]:
#Generate the links, save as pickle

In [54]:
PP = iolib.PrintProgress(maximum=sum([1 for x in range(1,17026,4)]), init_msg='Getting all page links ...')
for n in range(1,17026,4):
    s = "DoScroll('%s', 'PageHitList', '');" % n
    browser.execute_script(s)
    
    try:
        elements = WebDriverWait(browser, 10).until(ElementHasChanged(browser))
    except Exception as e:
        errs.append('Getting link for n=%s failed. The error was %s' % (n, e))
        PP.increment()
        continue
    
    links = get_links(elements)
    if links:
        ElementHasChanged.SCRAPED_LINKS.append(links)
        
    PP.increment()

all_links = set(ElementHasChanged.SCRAPED_LINKS)
if links:
    pickle.dump(file=links_pickle, obj=list(all_link))
    print('Saved links to %s' % links_pickle)
else:
    print('No links!!')

Getting all page links ...


KeyboardInterrupt: 

In [49]:
elements = WebDriverWait(browser, 10).until(ElementHasChanged(browser))

['https://www.actionlibrary.com/cgi-bin/emapwww?Page=DoPageSearch&Action=EditionPages&EditionURN=100665634&PageURN=134466950&FFAC=WM0000048174&r=-915574830&PageURN=134466950', 'https://www.actionlibrary.com/cgi-bin/emapwww?Page=DoPageSearch&Action=EditionPages&EditionURN=100665634&PageURN=134466951&FFAC=WM0000048174&r=-915559765&PageURN=134466951', 'https://www.actionlibrary.com/cgi-bin/emapwww?Page=DoPageSearch&Action=EditionPages&EditionURN=100665634&PageURN=134466952&FFAC=WM0000048174&r=-915544680&PageURN=134466952', 'https://www.actionlibrary.com/cgi-bin/emapwww?Page=DoPageSearch&Action=EditionPages&EditionURN=100665634&PageURN=134466953&FFAC=WM0000048174&r=-915529883&PageURN=134466953']
